In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
import numpy as np

In [3]:
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
from nltk.corpus import stopwords

In [5]:
from stop_words import get_stop_words
stop_words = get_stop_words('russian')

In [6]:
import re

In [7]:
ds = pd.read_csv("establishment_data.csv")

In [8]:
ds.dropna(subset=['description'], inplace=True)

In [9]:
ds['type_id'] = ds['type_id'].apply(lambda x: int(x))
ds['index'] = [int(i) for i in range(1,len(ds.index)+1)]
columns = ['index','title','city_id','type_id','address', 'phone',
           'ave_check','cuisine', 'payment',  'parking',
           'features', 'description',
           'schedule', 'tags_type', 'item_selections', 'lunch', 'restriction',
           'img1', 'img2', 'img3', 'img4','review1', 'review2', 'review3',
           'review4', 'review5', 'review6','user_name1', 'user_name2', 'user_name3', 'user_name4',
           'user_name5', 'user_name6']
ds = ds.reindex(columns=columns)

In [10]:
ds['user_name1']=ds['user_name1'].where(ds['user_name1'].isnull(), np.random.randint(1,6,ds.shape[0]))
ds['user_name2']=ds['user_name2'].where(ds['user_name2'].isnull(), np.random.randint(1,6,ds.shape[0]))
ds['user_name3']=ds['user_name3'].where(ds['user_name3'].isnull(), np.random.randint(1,6,ds.shape[0]))
ds['user_name4']=ds['user_name4'].where(ds['user_name4'].isnull(), np.random.randint(1,6,ds.shape[0]))
ds['user_name5']=ds['user_name5'].where(ds['user_name5'].isnull(), np.random.randint(1,6,ds.shape[0]))
ds['user_name6']=ds['user_name6'].where(ds['user_name6'].isnull(), np.random.randint(1,6,ds.shape[0]))


In [11]:
ds.loc[ds['user_name1'].notnull(), 'user_name1']=ds.loc[ds['user_name1'].notnull(), 'user_name1'].apply(int)
ds.loc[ds['user_name2'].notnull(), 'user_name2']=ds.loc[ds['user_name2'].notnull(), 'user_name2'].apply(int)
ds.loc[ds['user_name3'].notnull(), 'user_name3']=ds.loc[ds['user_name3'].notnull(), 'user_name3'].apply(int)
ds.loc[ds['user_name4'].notnull(), 'user_name4']=ds.loc[ds['user_name4'].notnull(), 'user_name4'].apply(int)
ds.loc[ds['user_name5'].notnull(), 'user_name5']=ds.loc[ds['user_name5'].notnull(), 'user_name5'].apply(int)
ds.loc[ds['user_name6'].notnull(), 'user_name6']=ds.loc[ds['user_name6'].notnull(), 'user_name6'].apply(int)
# ds[['user_name2']] = ds[~ds['user_name2'].isnull()][['user_name2']].astype(int)
# ds[['user_name3']] = ds[~ds['user_name3'].isnull()][['user_name3']].astype(int)
# ds[['user_name4']] = ds[~ds['user_name4'].isnull()][['user_name4']].astype(int)
# ds[['user_name5']] = ds[~ds['user_name5'].isnull()][['user_name5']].astype(int)
# ds[['user_name6']] = ds[~ds['user_name6'].isnull()][['user_name6']].astype(int)

In [12]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176 entries, 0 to 2113
Data columns (total 33 columns):
index              1176 non-null int64
title              1176 non-null object
city_id            1176 non-null int64
type_id            1176 non-null int64
address            1176 non-null object
phone              1172 non-null object
ave_check          1172 non-null object
cuisine            1176 non-null object
payment            1176 non-null object
parking            1176 non-null object
features           1175 non-null object
description        1176 non-null object
schedule           1176 non-null object
tags_type          708 non-null object
item_selections    283 non-null object
lunch              309 non-null object
restriction        25 non-null object
img1               1176 non-null object
img2               1176 non-null object
img3               1141 non-null object
img4               1090 non-null object
review1            365 non-null object
review2            201

In [13]:
ds.to_csv('final_data.csv',index=False)

In [60]:
content = ds[['address', 'cuisine','description','features','item_selections',
              'parking','tags_type']]
content.head()

,address,cuisine,description,features,item_selections,parking,tags_type
0,"г. Алматы, ул. Омаровой, 41/2 (Кок Тобе)",Грузинская,Грузинский ресторан Sololaki! Сололаки трудно ...,"Открывается красивый вид, Кальян, Есть летняя ...",Располагается на Кок Тобе,"Охраняемая бесплатная, на 80 маш.","Ресторан грузинской кухни,Панорамный вид,Кок-Т..."
3,"Алматы, Мамыр 7-й микрорайон, 8Б (Момышулы - А...",Европейская,Приглашаем посетить уютное семейное кафе «Жу-Ж...,"WiFi, ТВ-экран, Детские стульчики, Детская пло...","Есть детский уголок,Вкусно и недорого",Неохраняемая бесплатная,NaN
6,"г. Алматы, ул. Сейфуллина, 52 (уг. Жандосова, ...","Узбекская, Восточная","Дом Восточной кухни ""Навруз"". При создании диз...","Фоновая музыка, Есть летняя терасса, Топчаны, ...",NaN,Неохраняемая бесплатная,NaN
7,"г. Алматы, ул. Макатаева, 158 (уг. Байтурсынова)","Восточная, Европейская",Кальянный бар Khan El-Khalili,"WiFi, Кальян, Настольные игры",Где покурить кальян,Охраняемая бесплатная,"Lounge bar,Покурить кальян,Поиграть в настольн..."
8,"г. Алматы, Кабанбай батыра, 140 (уг. Сейфуллина)","Азиатская, Китайская",№1 в Алматы 🏆,"На вынос, Есть доставка",NaN,Городская,"пр. Сейфуллина,ул. Кабанбай батыра"


In [61]:
content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176 entries, 0 to 2113
Data columns (total 7 columns):
address            1176 non-null object
cuisine            1176 non-null object
description        1176 non-null object
features           1175 non-null object
item_selections    283 non-null object
parking            1176 non-null object
tags_type          708 non-null object
dtypes: object(7)
memory usage: 73.5+ KB


In [62]:
content["address"] = content["address"].str.replace(" ул.", "")
content["address"] = content["address"].str.replace("ул. ", "")
content["address"] = content["address"].str.replace(" уг.", "")
content["address"] = content["address"].str.replace("уг. ", "")
content["address"] = content["address"].str.replace("мкрн.", "")
content["address"] = content["address"].str.replace("р/н.", "")
content["address"] = content["address"].str.replace("\d+", "")
content["address"] = content["address"].str.replace(" ш. ", "")
content["address"] = content["address"].str.replace("/", "")
content["address"] = content["address"].str.replace("(", "")
content["address"] = content["address"].str.replace(")", "")
content["address"] = content["address"].str.replace("-й", "")
content["address"] = content["address"].str.replace("г. Алматы,", "")
content["address"] = content["address"].str.replace("г.Алматы,", "")
content["address"] = content["address"].str.replace("пр. ", "")
content["address"] = content["address"].str.replace("микрорайон", "")
content["address"] = content["address"].str.replace("км. ", "")
content["address"] = content["address"].str.replace("-", "")
content["address"] = content["address"].str.replace(" а", "")
content["address"] = content["address"].str.replace(" ", "")
content["address"] = content["address"].map(lambda x: str(x).split(','))

content

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

,address,cuisine,description,features,item_selections,parking,tags_type
0,"[Омаровой, КокТобе]",Грузинская,Грузинский ресторан Sololaki! Сололаки трудно ...,"Открывается красивый вид, Кальян, Есть летняя ...",Располагается на Кок Тобе,"Охраняемая бесплатная, на 80 маш.","Ресторан грузинской кухни,Панорамный вид,Кок-Т..."
3,"[Алматы, Мамыр, БМомышАбая]",Европейская,Приглашаем посетить уютное семейное кафе «Жу-Ж...,"WiFi, ТВ-экран, Детские стульчики, Детская пло...","Есть детский уголок,Вкусно и недорого",Неохраняемая бесплатная,NaN
6,"[Сейфуллина, Жандосова, Каменка]","Узбекская, Восточная","Дом Восточной кухни ""Навруз"". При создании диз...","Фоновая музыка, Есть летняя терасса, Топчаны, ...",NaN,Неохраняемая бесплатная,NaN
7,"[Макатаева, Байтурсынова]","Восточная, Европейская",Кальянный бар Khan El-Khalili,"WiFi, Кальян, Настольные игры",Где покурить кальян,Охраняемая бесплатная,"Lounge bar,Покурить кальян,Поиграть в настольн..."
8,"[Кабанбайбатыра, Сейфуллина]","Азиатская, Китайская",№1 в Алматы 🏆,"На вынос, Есть доставка",NaN,Городская,"пр. Сейфуллина,ул. Кабанбай батыра"
11,"[Абая, БЦентральныйстадион]",Европейская,Дорогие болельщики FC Kairat,"WiFi, Спортивные трансляции, Есть летняя терасса",NaN,Неохраняемая бесплатная,"Спортбар,пр. Абая"
13,"[​Айнабулак, ]","Восточная, Европейская","Семейное кафе ""Bugs Bunny""!","Детская площадка, Детский уголок, Детское меню...",NaN,Неохраняемая бесплатная,"Детский ресторан,Детские праздники,Развлечения..."
14,"[Ауэзова, вКабанбайбатыра]","Американская, Европейская",Территория вкусных стейков 🥩,"Фоновая музыка, Живая музыка, Танцпол, Бизнес-...",NaN,Неохраняемая бесплатная,"Мясной ресторан,Стейки,Банкеты,Зал на 120 перс..."
15,"[Райымбека, Тлендиева]","Корейская, Европейская",Корейский ресторан Пусан,"На вынос, Есть летняя терасса",NaN,Городская,"Ресторан корейской кухни,Вкусно и недорого,пр...."
16,"[Тулебаева, Макатаева]","Европейская, Итальянская",Живая музыка ежедневно. В праздничные дни шоу-...,"WiFi, Есть доставка, Фоновая музыка, Живая муз...","Среди самых популярных баров Алматы 2019 года,...",Неохраняемая бесплатная,"Пивной паб,Стейки,Коктейли,Выступления Live-гр..."


In [63]:
content["tags_type"] = content["tags_type"].str.replace(" ул.", "")
content["tags_type"] = content["tags_type"].str.replace("ул. ", "")
content["tags_type"] = content["tags_type"].str.replace(" уг.", "")
content["tags_type"] = content["tags_type"].str.replace("уг. ", "")
content["tags_type"] = content["tags_type"].str.replace("мкрн.", "")
content["tags_type"] = content["tags_type"].str.replace("р/н.", "")
content["tags_type"] = content["tags_type"].str.replace("\d+", "")
content["tags_type"] = content["tags_type"].str.replace(" ш.", "")
content["tags_type"] = content["tags_type"].str.replace("/", "")
content["tags_type"] = content["tags_type"].str.replace("(", "")
content["tags_type"] = content["tags_type"].str.replace(")", "")
content["tags_type"] = content["tags_type"].str.replace("-й", "")
content["tags_type"] = content["tags_type"].str.replace("г. Алматы,", "")
content["tags_type"] = content["tags_type"].str.replace("г.Алматы,", "")
content["tags_type"] = content["tags_type"].str.replace("пр. ", "")
content["tags_type"] = content["tags_type"].str.replace("микрорайон", "")
content["tags_type"] = content["tags_type"].str.replace("км. ", "")
content["tags_type"] = content["tags_type"].str.replace("-", " ")
content["tags_type"] = content["tags_type"].str.replace(" а ", "")
content["tags_type"] = content["tags_type"].str.replace(" ", "")
content["tags_type"] = content["tags_type"].map(lambda x: str(x).split(','))


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

In [64]:
# content["description"] = content["description"].str.replace(",", " ")
# content["description"] = content["description"].str.replace("-", " ")

# content

In [65]:
content["cuisine"] = content["cuisine"].map(lambda x: str(x).split(','))

content["features"] = content["features"].str.replace("WIFI", "ВАЙФАЙ")
content["features"] = content["features"].str.replace("WiFi", "ВАЙФАЙ")
content["features"] = content["features"].str.replace("VIP", "ВИП")
content["features"] = content["features"].str.replace(" ", "")
content["features"] = content["features"].map(lambda x: str(x).split(','))

content["parking"] = content["parking"].str.replace(", на", "")
content["parking"] = content["parking"].str.replace("\d+", "")
content["parking"] = content["parking"].str.replace("маш.", "")
content["parking"] = content["parking"].str.replace(" ", "")
content["parking"] = content["parking"].map(lambda x: str(x).split(','))


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [66]:
content.drop(columns=['item_selections'],inplace=True)

In [67]:
content['description'] = content['description'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [68]:
def get_keywords(x):
    r = Rake(stop_words)
    r.extract_keywords_from_text(x)
    return r.get_ranked_phrases()

content['keywords'] = content['description'].apply(lambda x: get_keywords(x))

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [69]:
content.drop(columns = ['description'], inplace = True)
content['bag_of_words'] = ''
columns = content.columns
for index, row in content.iterrows():
    words = ''
    
    for col in columns:
        words = words + ' '.join(row[col])+ ' '
        
    row['bag_of_words'] = words
    
content.drop(columns = [col for col in content.columns if col!= 'bag_of_words'], inplace = True)

content['city_id'] = ds['city_id'].values
content['type_id'] = ds['type_id'].values
content['index'] = ds['index'].values
content['title'] = ds['title'].values

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [223]:
def recommendation_from_history(city_id, type_id, title, content=content):
    
    recommended = []

    filter_by_city = content[content['city_id']==city_id]
    filter_by_type = filter_by_city[filter_by_city['type_id']==type_id]
    index_of_title = filter_by_type[filter_by_type['title']==title]['index']
    ind = list(filter_by_type['index'].index).index(index_of_title.index)

    count = CountVectorizer()
    count_matrix = count.fit_transform(filter_by_type['bag_of_words'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)

    score_series = pd.Series(cosine_sim[ind]).sort_values(ascending = False)

    top_10 = list(score_series.iloc[1:11].index)
    for i in top_10:
        recommended.append(content[content.index==list(filter_by_type['index'].index)[i]]['index'].get_values()[0])
        
    return recommended

In [224]:
recommendation_from_history(1,1,'Brewery')

[1082, 831, 1147, 1096, 1019, 1025, 953, 1058, 1108, 975]

In [72]:
#testing
ds[ds['index']==735] 

,index,title,city_id,type_id,address,phone,ave_check,cuisine,payment,parking,...,review3,review4,review5,review6,user_name1,user_name2,user_name3,user_name4,user_name5,user_name6
1390,735,Sea Marine,1,1,"г. Алматы, пр. Достык, 123/8",+7 (777) 225‒44‒36,7000-10000 тг,"Морепродукты, Средиземноморская","Наличные, Оплата картой",Неохраняемая бесплатная,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##  Second function

In [248]:
# content.drop([2114],inplace =True)
# content

In [246]:
def recommendations_preference(city_id, type_id, preference_list, content=content):
    
#     content.loc[content.index[-1]+1] = ['Восточный Кальян ВАЙФАЙ Неохроняемая бесплатная', 1, 3, len(content['index'])+1, str(len(content['index'])+1)]
    content.loc[content.index[-1]+1] = [preference_list, city_id, type_id, len(content['index'])+1, str(len(content['index'])+1)]

    recommended = []

    filter_by_city = content[content['city_id']==city_id]
    filter_by_type = filter_by_city[filter_by_city['type_id']==type_id]
    index_of_title = filter_by_type[filter_by_type['title']==str(content['index'].iat[-1])]['index'] 
    ind = list(filter_by_type['index'].index).index(index_of_title.index)

    count = CountVectorizer()
    count_matrix = count.fit_transform(filter_by_type['bag_of_words'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)

    score_series = pd.Series(cosine_sim[ind]).sort_values(ascending = False)

    top_10 = list(score_series.iloc[1:11].index)
    
    for i in top_10:
        recommended.append(content[content.index==list(filter_by_type['index'].index)[i]]['index'].get_values()[0])
        
    return recommended

In [247]:
recommendations_preference(1, 3, 'Восточный Кальян ВАЙФАЙ Неохроняемая бесплатная')

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


[236, 263, 293, 150, 472, 700, 492, 538, 138, 814]